#  Analise de sentimentos

* Análise de sentimentos com algoritmo de VADER

* Utilizando TF-IDF para cálculo de similaridade (base para criação de um chatbot).

* Nós podemos usar os tokens extraídos das palavra, n-gramse as técninas que vimos até o momento para construir um sistema de análise de sentimentos (uma das principais aplicações em NLP).

* A análise de sentimentos permite saber, por exemplo, o que as pessoas pensam sobre determinada entidade (produto, musica, game, filme, etc).

* A maioria dos produtos são classificados por estrelas (e-commerce por exemplo)

* No entanto, um forma mais natural é usar os comentários (muitas das plataformas de e-commerce possuem) em linguagem natural.

* O grande problema no uso de comentários para saber o feedback de um produto é que será necessário ler os comentários para uma análise.

* Um piepline de NLP pode processar uma grande quantidade de feedback do usuário de maneira rápida e objetiva, eleminando um possível viés que a análise humana possa ter.

* E um pipeline de NLP pode gerar uma classificação numérica sobre o comentário (negativo ou positivo).

* Outra aplicação é a análise de mensagem indesejada e inadequadas para determinado contexto.

* Um chatbot, por exemplo, pode ser capaz de medir o sentimento nas mensagens de chat processadas para responder adquadamente (ou analisar suas próprias mensagens antes de responder).

* Suponha que o objetivo seja medir o quão positivo um texto pode ser (texto sobre um determinado produto).

* Este algoritmo pode produzir um número de ponto flutuante entre +1 e -1 (+1 para texto com sentimentos positivos como: "Absolutamente perfeito! Adoro! :-)""...

* E -1 para texto com sentimento negativos como "Horrível!!!, Completamente inútil :(".

* Existem duas abordagens para análise de sentimentos:

    * Um algoritmo de regras compostas por um ser humano.

    * Um modelo de machine learning treinado com dados produzidos por computador.


* A primeira abordagem usa regras criadas por seres humanos para medir sentimentos.

* Uma abordagem comum é encontrar palavras-chaves no texto e mapear cada uma para pontuações ou pesos numéricos em um dicionário, por exemplo.

* Neste caso, usamos tokenização, stems, lemas ou n-grams no dicionário, em vez de apenas palavras.

* A "regra" do algoritmo seria somar essas pontuações para cada palavra-chave em um documento encontradas no dicionário de pontuações.

* Podemos usar o algoritmos VADER (sklearn) para produzir este dicionário de pontuações.

## Algoritmo de Vader

* Criado por Hutto E Gilbert na GA Tech, o VADER (valence Aware Dictionary For sEntiment Reasoning) é um dos primeiros algoritmos baseados em regras.

* Muitos dos pacotes de NLP (NLTK) implementam este algoritmo.

* Para instalação em python: "pip install vaderSentiment"

In [1]:
!pip install vaderSentiment

In [2]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

sa = SentimentIntensityAnalyzer()
print(sa.lexicon)

.1, 'stupid': -2.4, 'stupider': -2.5, 'stupidest': -2.4, 'stupidities': -2.0, 'stupidity': -1.9, 'stupidly': -2.0, 'stupidness': -1.7, 'stupidnesses': -2.6, 'stupids': -2.3, 'stutter': -1.0, 'stuttered': -0.9, 'stutterer': -1.0, 'stutterers': -1.1, 'stuttering': -1.3, 'stutters': -1.0, 'suave': 2.0, 'submissive': -1.3, 'submissively': -1.0, 'submissiveness': -0.7, 'substantial': 0.8, 'subversive': -0.9, 'succeed': 2.2, 'succeeded': 1.8, 'succeeder': 1.2, 'succeeders': 1.3, 'succeeding': 2.2, 'succeeds': 2.2, 'success': 2.7, 'successes': 2.6, 'successful': 2.8, 'successfully': 2.2, 'successfulness': 2.7, 'succession': 0.8, 'successional': 0.9, 'successionally': 1.1, 'successions': 0.1, 'successive': 1.1, 'successively': 0.9, 'successiveness': 1.0, 'successor': 0.9, 'successors': 1.1, 'suck': -1.9, 'sucked': -2.0, 'sucker': -2.4, 'suckered': -2.0, 'suckering': -2.1, 'suckers': -2.3, 'sucks': -1.5, 'sucky': -1.9, 'suffer': -2.5, 'suffered': -2.2, 'sufferer': -2.0, 'sufferers': -2.4, 'suff

In [3]:
score = [(tok, score) for tok, score in sa.lexicon.items() if " " in tok]
print(score)

[("( '}{' )", 1.6), ("can't stand", -2.0), ('fed up', -1.8), ('screwed up', -1.5)]


In [4]:
print(len(sa.lexicon))

7504


In [5]:
print(sa.polarity_scores(text="Python is very readble and it's great for NLP"))

{'neg': 0.0, 'neu': 0.661, 'pos': 0.339, 'compound': 0.6249}


In [6]:
print(sa.polarity_scores(text="Python is not a bad for most application."))

{'neg': 0.0, 'neu': 0.711, 'pos': 0.289, 'compound': 0.431}


In [7]:
print(sa.polarity_scores(text="Python is terrible."))

{'neg': 0.608, 'neu': 0.392, 'pos': 0.0, 'compound': -0.4767}


In [8]:
corpus = ["Absolutely perfect! Love it! :) XD", "Horrible! Completely useless :(", "It was OK, Some good and some bad things"]

In [9]:
for doc in corpus:
    scores = sa.polarity_scores(doc)
    print("{:+}: {}".format(scores['compound'], doc))

+0.9558: Absolutely perfect! Love it! :) XD
-0.8768: Horrible! Completely useless :(
-0.1531: It was OK, Some good and some bad things


## Ranking



In [18]:
from nltk.tokenize import TreebankWordTokenizer
from collections import Counter
import copy
import math

tokenizer = TreebankWordTokenizer()

docs = ["The faster Harry got to the store, the faster and faster Harry would get home."]
docs.append("Harry is hairy and faster than Jill.")
docs.append("Jill is not as hairy as Harry.")

doc_tokens = []
for doc in docs:
    doc_tokens += [sorted(tokenizer.tokenize(doc.lower()))]

all_doc_tokens = sum(doc_tokens, [])
lexicon = sorted(all_doc_tokens)

def cosine_sim(vec1, vec2):
    vec1 = [val for val in vec1.values()]
    vec2 = [val for val in vec2.values()]


    dot_prod = 0
    for i, v in enumerate(vec1):
        dot_prod += v * vec2[i]

    mag_1 = math.sqrt(sum([x**2 for x in vec1]))
    mag_2 = math.sqrt(sum([x**2 for x in vec2]))

    return dot_prod / (mag_1 * mag_2)

In [19]:
from collections import OrderedDict

zero_vector = OrderedDict((token, 0) for token in lexicon)
document_tfidf_vectors = []
for doc in docs:    
    vec = copy.copy(zero_vector)    
    tokens = tokenizer.tokenize(doc.lower())    
    token_counts = Counter(tokens)    
    
    for key, value in token_counts.items():        
        docs_containing_key = 0        
        for _doc in docs:            
            if key in _doc:                
                docs_containing_key += 1        
            tf = value / len(lexicon)        
            if docs_containing_key:            
                idf = len(docs) / docs_containing_key        
            else:            
                idf = 0        
            vec[key] = tf * idf    
        document_tfidf_vectors.append(vec)

In [20]:
query = "How long does it take to get to the store?"
query_vec = copy.copy(zero_vector)

tokens = tokenizer.tokenize(query.lower())
token_counts = Counter(tokens)

for key, value in token_counts.items():
    doc_contains_key = 0
    for _doc in docs:
        if key in _doc:
            docs_containing_key += 1
        tf = value / len(lexicon)
        if doc_contains_key:
            idf = len(docs) / doc_contains_key
        else:
            idf = 0
        query_vec[key] = tf * idf
    document_tfidf_vectors.append(query_vec)

print(document_tfidf_vectors[0])

OrderedDict([(',', 0.09090909090909091), ('.', 0.030303030303030304), ('and', 0.045454545454545456), ('as', 0), ('faster', 0.13636363636363635), ('get', 0.09090909090909091), ('got', 0.09090909090909091), ('hairy', 0), ('harry', 0.0), ('home', 0.09090909090909091), ('is', 0), ('jill', 0), ('not', 0), ('store', 0.09090909090909091), ('than', 0), ('the', 0.2727272727272727), ('to', 0.09090909090909091), ('would', 0.09090909090909091)])


* Assim, temos uma representação vetorial k-dimensional (K é o número de palavras do vocabulário) de cada documento no corpus.

* Podemos dizer que dois vetores, em um determinado espaço vetorial, são similares se tiverem um ângula semelhante.

* Considerando cada vetor começando na origem (0, ...,0), os que alcançarem o mesmo ângulo são similares, mes que não alcancem a mesma distância (comprimento do vetor no espaço Euclideano).

* Dois vetores são considerados similares se a similaridade do cosseno for alta; portanto, você pode encontrar a similaridade entre dois vetores se estes minimizarem a função:

    * $\cos \Theta = \frac{A . B}{|A| . |B|}$

* Para calcular a similaridade entre dois documentos, podemos tratar a consulta em si como um documento e, portanto, obter a representação vetorial baseada em TF-IDF.

* A última etapa é encontrar os documentos cujos vetores têm os maiores valores de similaridades da distância de cosseno com a consulta e retorna-los como resultado da pesquisa.

In [21]:
query = 'How long does it take to get to the store ?'
query_vec  = copy.copy(zero_vector)

tokens = tokenizer.tokenize(query.lower())
token_counts = Counter(tokens)

for key, valule in token_counts.items():
    doc_contains_key = 0
    for _doc in docs:
        if key in _doc.lower():
            doc_contains_key += 1
        if doc_contains_key == 0:
            continue
        tf = value / len(tokens)
        idf = len(docs) / doc_contains_key
        query_vec[key] = tf * idf

In [26]:
token_counts

Counter({'how': 1,
         'long': 1,
         'does': 1,
         'it': 1,
         'take': 1,
         'to': 2,
         'get': 1,
         'the': 1,
         'store': 1,
         '?': 1})

In [24]:
print(cosine_sim(query_vec, document_tfidf_vectors[0]))
print(cosine_sim(query_vec, document_tfidf_vectors[1]))
print(cosine_sim(query_vec, document_tfidf_vectors[2]))

0.6954006683576303
0.6954006683576303
0.6954006683576303


* O doc 0 possui a maior relevância para a consulta. E assim, é possível encontrar documentos relevantes em qualquer corpus, sejam artigos na Wikipedia ou tweets.

* Observe que o mecanismo de busca do Google funciona de forma diferente, baseado em índices dos vetores TF-IDF (não é o nosso objetivo aqui).

* A maioria dos mecanismos de pesquisa pode responder em tempo constante (0 (1)) porque eles usam um índice invertido.

* Para saber mais sobre índices:

    * https://pypi.org/project/Whoosh/

    * https://github.com/Mplsbeb/whoosh

* Esa aplicação de busca por similaridade é uma etapa importante de um pipeline para NLP.

* Alguns chatbots dependem exclusivamente de um mecanismo de busca como seu único algoritmo para gerar respostas.

* Você precisa executar uma etapa adicional para transformar seu indice de pesquisa simples (TF-IDF) em um chatbot.

* Uma possibilidade para o chatbot, é necessario armazenar dados de treinamento em pares de perguntas e respostas apropriadas.

* Em seguida, calculamos o TF-IDF para pesquisar uma pergunta mais semelhante com o texto de entrada do usuário.

* Em vez de retornar a instrução mais similares de um banco de dados, retornamos a resposta associada a esta intrução.